<a href="https://colab.research.google.com/github/h0806449f/PyTorch/blob/main/DB_01_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch workflow

1. Data preprocessing
2. Build the model
    2.1 Pick loss function & optimizer
    2.2 Build training loop
3. Fit model to data
4. Make prediction
5. Evaluate the model
6. Improve the model (through experimentation)
7. Save the model (for reload)


In [5]:
import torch
from torch import nn  # nn contains all of PyTorch's buinding blocksof CNN
# https://pytorch.org/docs/stable/nn.html
import matplotlib.pyplot as plt

# check version
torch.__version__

'2.0.1+cu118'

# Data (preparing and loading)

In [6]:
# Create known parameters
weight = 0.7
bias = 0.3

# Create
start = 0
end = 1
step = 0.02

# Input
X = torch.arange(start, end, step).unsqueeze(dim=1)
# Output
y = weight * X + bias

# 取前 10 檢視
X[:10], y[:10]

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]))

# Data preparing : splitting data into training and test sets
1. training set
2. validation set
3. test set

In [ ]:
# Create train & test split

# 算出訓練資料的比例
train_split = int(0.8 * len(X))

# 分割問題與答案
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

# 檢查資料筆數
X_train.shape, y_train.shape, X_test.shape, y_test.shape

(torch.Size([40, 1]),
 torch.Size([40, 1]),
 torch.Size([10, 1]),
 torch.Size([10, 1]))

# Build model
1. more info: https://pytorch.org/docs/stable/generated/torch.nn.Module.html
2. Pytorch model 元素
    1. torch.nn - contains all of the buildings for neural networl
    2. torch.nn.Parameter - 模型要學習的參數 (通常模型會自動設置)
    3. torch.nn.Module - 神經網絡基本款, if subclass it, should overwrite forward() 前向傳播過程
    4. torch.optim - optimizers
    5. def forward() - All nn.Muodule subclass require to overwrite def forward() 前向傳播
    6. Doc: https://pytorch.org/tutorials/beginner/ptcheat.html

In [ ]:
# Create liner regression model class
from torch import nn


class LinerRegressionModel(nn.Module):
# subclass nn.Module
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1,
                              # start with random weight
                              requires_grad=True,
                              # can this parameter be updated via gradient?
                              dtype=torch.float))
                              # ML DL prefer float

        self.bias = nn.Parameter(torch.randn(1,
                             # start with random bias
                             requires_grad=True,
                             dtype=torch.float))

    # Forward method to define the computation in the model
    def forward(self, x:torch.Tensor) -> torch.Tensor:
    # x is the input dara
        return self.weights * x + self.bias
        # this is the linear regression formula

# Check the contents of model

In [ ]:
# Create random seed
torch.manual_seed(42)

# Create an nn.Module
model_0 = LinerRegressionModel()

# Check
model_0.parameters()   # <generator object Module.parameters at 0x7f972b1ed770>

print("清單形式: ", list(model_0.parameters()) )  # 以list列出元素

print("名稱形式: ", model_0.state_dict())

清單形式:  [Parameter containing:
tensor([0.3367], requires_grad=True), Parameter containing:
tensor([0.1288], requires_grad=True)]
名稱形式:  OrderedDict([('weights', tensor([0.3367])), ('bias', tensor([0.1288]))])


# Make prediction
torch.inference_mode()

In [ ]:
with torch.inference_mode():
# inference_mode 會關掉 requires_grad=True 因為這邊用於簡單的顯示資料
    y_preds = model_0(X_test)

# 同效果
with torch.no_grad():
    y_preds = model_0(X_test)

# Train model
1. 訓練模型
2. 減少loss
3. info: https://pytorch.org/docs/stable/nn.html#loss-functions
4. 選擇 loss function 損失函式 (需要依照問題選擇)
5. 選擇 optimizer 優化器

In [ ]:
# Setup loss function
loss_fn = nn.L1Loss()
loss_fn   # L1Loss()

# Setup an optimizer (stochastic gradient descent)
optimizer = torch.optim.SGD(params=model_0.parameters(),
                # 要優化的參數 是之前的參數產生器
                lr=0.01,
                # learning rate
                )

# Building training loop and testing loop
1. Training loop
    1. Loop through the data
    2. Forward pass
    3. Calculate the loss
    4. Optimizer zero grad
    5. Loss backward
    6. Optimizer step - use optimizer to adjust model's parameters and then try to improve the loss

In [ ]:
# 1. Loop through the data
# epochs how many loop through the data
epochs = 201

for epoch in range(epochs):
    # Set the model to training mode
    model_0.train()

    # 2. Forward pass
    y_pred = model_0(X_train)

    # 3. Calculate the loss
    loss = loss_fn(y_pred, y_train)

    # 4. Optimizer zero grad
    # 使用 optimizer.zero_grad() 將之前儲存的 gradient 歸零，避免累積的 gradient 影響 optimizer 更新參數的方向。
    optimizer.zero_grad()

    # 5. Perform backward base on the loss
    loss.backward()

    # 6. Step the optimizer (perform gradient descent)
    # by default, how the optimizer change will acculumate through the loop
    optimizer.step()

    # TESTING !!!
    model_0.eval()
    with torch.inference_mode():
        # 1. Forward pass
        test_pred = model_0(X_test)

        # 2. Calculate the loss
        test_loss = loss_fn(test_pred, y_test)

    if epoch % 20 == 0:
        print("=" * 50)
        print(f"Epoch: {epoch} | Loss: {loss} | Test_loss: {test_loss}")
        print(model_0.state_dict())


# epoch 180 之後 停止下降 再繼續訓練可能會導致過擬合

Epoch: 0 | Loss: 0.31288138031959534 | Test_loss: 0.48106518387794495
OrderedDict([('weights', tensor([0.3406])), ('bias', tensor([0.1388]))])
Epoch: 20 | Loss: 0.08908725529909134 | Test_loss: 0.21729660034179688
OrderedDict([('weights', tensor([0.4184])), ('bias', tensor([0.3333]))])
Epoch: 40 | Loss: 0.04543796554207802 | Test_loss: 0.11360953003168106
OrderedDict([('weights', tensor([0.4748])), ('bias', tensor([0.3868]))])
Epoch: 60 | Loss: 0.03818932920694351 | Test_loss: 0.08886633068323135
OrderedDict([('weights', tensor([0.5116])), ('bias', tensor([0.3788]))])
Epoch: 80 | Loss: 0.03132382780313492 | Test_loss: 0.07232122868299484
OrderedDict([('weights', tensor([0.5459])), ('bias', tensor([0.3648]))])
Epoch: 100 | Loss: 0.024458957836031914 | Test_loss: 0.05646304413676262
OrderedDict([('weights', tensor([0.5800])), ('bias', tensor([0.3503]))])
Epoch: 120 | Loss: 0.01758546568453312 | Test_loss: 0.04060482233762741
OrderedDict([('weights', tensor([0.6141])), ('bias', tensor([0.

In [ ]:
model_0.state_dict()
# 每run run 一次 epoch -> loss 就會減少
# 原本 : # OrderedDict([('weights', tensor([0.3367])), ('bias', tensor([0.1288]))])
# OrderedDict([('weights', tensor([0.3406])), ('bias', tensor([0.1388]))])
# OrderedDict([('weights', tensor([0.3445])), ('bias', tensor([0.1488]))])
# OrderedDict([('weights', tensor([0.3484])), ('bias', tensor([0.1588]))])
# ....

OrderedDict([('weights', tensor([0.6951])), ('bias', tensor([0.2993]))])

# Save the model
1. torch.save() - save Pytorch object
2. torch.load() - load Pytorch object
3. torch.nn.Module.load_state_dict() - load state_dict

In [ ]:
# Saving
from pathlib import Path

# 1. Create directory
MODEL_PATH = Path("model")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path
MODEL_NAME = "01_pytorch_workflow_model_0.pt"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. Save the model state dict
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model_0.state_dict(),
      f=MODEL_SAVE_PATH)

Saving model to: model/01_pytorch_workflow_model_0.pt


In [ ]:
# Loading
# 上方僅儲存 state_dict
loaded_model_0 = LinerRegressionModel()

loaded_model_0.load_state_dict(torch.load(f=MODEL_SAVE_PATH))

<All keys matched successfully>

In [ ]:
loaded_model_0.state_dict()

OrderedDict([('weights', tensor([0.6951])), ('bias', tensor([0.2993]))])

In [ ]:
# Predict again
loaded_model_0.eval()

with torch.inference_mode():
    loaded_model_preds = loaded_model_0(X_test)

loaded_model_preds

tensor([[0.8554],
        [0.8693],
        [0.8832],
        [0.8971],
        [0.9110],
        [0.9249],
        [0.9388],
        [0.9527],
        [0.9666],
        [0.9805]])